### Simple voltage divider
We are using a simple resistor voltage divider circuit (`dut.cir`) to introduce Ngspice and the py4spice Python package. The circuit is a DC voltage supply (`V1`) connected across two resistors (`R1 R2`) in series. The node (`out`) between the resistor is the output.

Ngspice can be run in a number of different ways. The method used in this example is ***opinionated***. When using py4spice in a Python script/Jupyter Notebook, Ngspice is run in batch mode, reading a single top-level netlist. The netlist contains everything Ngspice needs for simulation.

When using py4spice the script will typically entail six steps:
1. Set up
1. Contruct control section
1. Contruct top netlist
1. Execute Ngspice simulation
1. Convert simulation results to more usable format
1. Analysis and output

More advanced analysis may involve looping through steps 2 through 6 while varying component values or circuit topology.

### Imports
Note `py4spice` is renamed to `spi`

In [1]:
from pathlib import Path
import py4spice as spi

### Set up paths dictionary
There are somes paths to define. They're put into a dictionary to keep things organized. Three subdirectories are defined:
- `examples/divider/netlists`
- `examples/divider/python`
- `examples/divider/results`

In [2]:
paths_dict: dict[str, Path] = {}           # define empty paths dictionary
paths_dict["project"] = Path("/workspaces/learn_ngspice/examples/divider")
paths_dict["netlists"] = paths_dict["project"] / "netlists"
paths_dict["results"] = paths_dict["project"] / "results"
paths_dict["results"].mkdir(parents=True, exist_ok=True) # create results directory if it does not exist

paths_dict["ngspice"] = Path("/usr/bin/ngspice") # ngspice executable

# create simulation transcript file. If it exists, make sure it is empty
sim_tran_filename: Path = paths_dict["results"] / "sim_transcript.log"
if sim_tran_filename.exists():  # delete and recreate. this makes sure it's empty
    sim_tran_filename.unlink()
sim_tran_filename.touch()

### Set up netlists dictionary
Netlist objects are created. These netlist snippets will be used later to create a single top-level netlist which will be read in during the batch simulation. Again, we put them into a dictionary to keep things organized.

In [3]:
netlists_dict: dict[str, spi.Netlist] = {}  # define empty netlists dictionary

# create these lines to use when concatenating to make full netlist
netlists_dict["blankline"] = spi.Netlist("")  # blank line for spacing
netlists_dict["title"] = spi.Netlist("* Simple Voltage Divider")
netlists_dict["end_line"] = spi.Netlist(".end")

# the main circuit ("device under test")
netlists_dict["dut"] = spi.Netlist(paths_dict["netlists"] / "dut.cir")

### Set up vectors dictionary
Vectors objects are used to define a set of circuit signals to create and display. Again, we put them into a dictionary to keep things organized.

In [4]:
vectors_dict: dict[str, spi.Vectors] = {}   # empty vectors dictionary

vectors_dict["vec_all"] = spi.Vectors("all")
vectors_dict["vec_in"] = spi.Vectors("in")
vectors_dict["vec_out"] = spi.Vectors("out")

### Set up analyses to run during the simulation process

Each batch simulation in Ngspice can perform one or more analyses. In this simple example, we will only be simulating a single analysis. We create a list of Analyses. The Analysis object has the following arguments:

| Argument | Description |
| :--- | :--- |
| `name` | Label given to the object |
| `cmd_type` | Ngspice has a different types of analyses. The `op`, `tran`, `ac`, `dc` analyses are supported with py4spice. |
| `cmd` | This is the actual command that will be sent to Ngspice. Its arguments can be found in the Ngspice documentation. |
| `vector` | This defines which signals will be in the simulation results. |
| `results_loc` | Where to put the results files. |

In [5]:
list_of_analyses: list[spi.Analyses] = []  # start with an empty list

# 1st analysis: operating point
op1 = spi.Analyses(
    name="op1",
    cmd_type="op",
    cmd="op",
    vector=vectors_dict["vec_all"],
    results_loc=paths_dict["results"],
)
list_of_analyses.append(op1)

# For this example there is only one analysis. If there were more, we would define and append. 

### Create control section
This is the section that tells ngspice what operations to do. A netlist object snippet is created. It will be concatenated with the other netlist objects to create the top-level netlist.

In [6]:
my_control = spi.Control()  # create 'my_control' object

# add all the analyses defined above into the control section
for analysis in list_of_analyses:
    my_control.insert_lines(analysis.lines_for_cntl())

# convert control section into a netlist object
netlists_dict["control1"] = spi.Netlist(str(my_control))

### Create a single netlist, ready for simulation
The netlist snippets are concatenated and merged into `top`. `top.cir` is written out. It will be read in during the simulation. We print out `top` to verify everything is correct.

In [7]:
# concatenate all tne netlists to make top1 and add to netlist dict
netlists_dict["top"] = (
    netlists_dict["title"]
    + netlists_dict["blankline"]
    + netlists_dict["dut"]
    + netlists_dict["blankline"]
    + netlists_dict["control1"]
    + netlists_dict["blankline"]
    + netlists_dict["end_line"]
    + netlists_dict["blankline"]
    )

# write netlist to a file so ngspice can read it
top_filename: Path = paths_dict["netlists"] / "top.cir"
netlists_dict["top"].write_to_file(top_filename)


# here is the netlist in top.cir and will be used for simulation
print(netlists_dict["top"])

* simple voltage divider

v1 in 0 dc 10
r2 out 0 2k
r1 in out 1k

.control
* timestamp: tue dec  2 23:26:27 2025
set wr_singlescale  $ makes one x-axis for wrdata
set wr_vecnames     $ puts names at top of columns
op
print line all > /workspaces/learn_ngspice/examples/divider/results/op1.txt
quit
.endc

.end



### Simulate
The Ngspice command is constructed, printed out for verification, and executed. `timeout` (in seconds) is set in case the simulation hangs.

In [8]:
# prepare simulate object and simulate
sim: spi.Simulate = spi.Simulate(
    ngspice_exe=paths_dict["ngspice"],
    netlist_filename=top_filename,
    transcript_filename=sim_tran_filename,
    name="sim1",
    timeout=5,
)
# print out the command that will be used in the spawned process
print(sim.ngspice_command)

sim.run()  # run the Ngspice simulation

['/usr/bin/ngspice', '-b', '/workspaces/learn_ngspice/examples/divider/netlists/top.cir']


### Convert simulation results to usable format
Ngspice simulation results are written out to a text file. These results are converted to a `SimResults` object for easier post-processing.

In [9]:
# There's a list of sim_results in case there was more than one analysis for the simulation.
#   In this case there is only one analysis performed during simulation
sim_results: list[spi.SimResults] = [
    spi.SimResults.from_file(analysis.cmd_type, analysis.results_filename)
    for analysis in list_of_analyses
]

### Print out op (operating point) results
The `print_section` function creates an easy-to-read table

In [11]:
# display results for operating point analysis
spi.print_section("Operating Point Results", sim_results[0].table_for_print())


--- Operating Point Results ---
in         10.000
out        6.667
v1#branch −3.333m

-------------------------------



### Final notes
This was a very simple circuit, with only a table of values and no post-processing. Other examples will be more complex. However, all of the scripts using py4spice will similarly follow these set of steps.